In [1]:
import os

In [2]:
import pandas 

In [3]:
import numpy as np

In [4]:
from shutil import copytree

In [5]:
from pyiron import Project

In [6]:
def getdir(path): 
    path_base_name = os.path.basename(path)
    if path_base_name == "":
        return os.path.basename(os.path.dirname(path))
    else: 
        return path_base_name

In [7]:
def update_id_lst(record_lst, job_id_lst):
    masterid_lst = []
    for masterid in record_lst:
        if np.isnan(masterid):
            masterid_lst.append(masterid)
        elif isinstance(masterid, int) or isinstance(masterid, float):
            masterid = int(masterid)
            masterid_lst.append(job_id_lst[masterid])
    return masterid_lst

In [8]:
def import_jobs(directory_to_import_to, archive_directory, df):
    archive_name = getdir(path=archive_directory)
    pr_import = Project(directory_to_import_to)
    
    # Copy HDF5 files
    copytree(archive_directory, directory_to_import_to, dirs_exist_ok=True) 
    
    # Update Database
    df["project"] = [os.path.join(pr_import.project_path, os.path.relpath(p, archive_name)) for p in df["project"].values]
    df['projectpath'] = len(df) * [pr_import.root_path]
    
    # Add jobs to database 
    job_id_lst = []
    for entry in df.to_dict(orient="records"):
        del entry['id']
        del entry['parentid']
        del entry['masterid']
        entry["timestart"] = pandas.to_datetime(entry["timestart"])
        entry["timestop"] = pandas.to_datetime(entry["timestop"])
        job_id = pr_import.db.add_item_dict(par_dict=entry)
        job_id_lst.append(job_id)
        
    # Update parent and master ids 
    for job_id, masterid, parentid in zip(job_id_lst, update_id_lst(record_lst=df['masterid'].values, job_id_lst=job_id_lst), update_id_lst(record_lst=df['parentid'].values, job_id_lst=job_id_lst)):
        if not np.isnan(masterid) or not np.isnan(parentid):
            pr_import.db.item_update(item_id=job_id, par_dict={'parentid': parentid, 'masterid': masterid})

In [9]:
df = pandas.read_csv("export.csv", index_col=0)
df

,id,status,chemicalformula,job,subjob,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,0,finished,Al,murn,/murn,archive/murn,2020-11-10 10:13:29.972786,2020-11-10 10:14:00.727395,30.0,pyiron@MacBook-Pro.local#1#11/11,Murnaghan,0.3.0,NaN,NaN
1,1,finished,Al,strain_0_9,/strain_0_9,archive/murn/murn_hdf5,2020-11-10 10:13:30.989205,2020-11-10 10:13:31.635240,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,NaN,0.0
2,2,finished,Al,strain_0_92,/strain_0_92,archive/murn/murn_hdf5,2020-11-10 10:13:32.900224,2020-11-10 10:13:33.879257,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,NaN,0.0
3,3,finished,Al,strain_0_94,/strain_0_94,archive/murn/murn_hdf5,2020-11-10 10:13:35.903589,2020-11-10 10:13:36.545234,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,NaN,0.0
4,4,finished,Al,strain_0_96,/strain_0_96,archive/murn/murn_hdf5,2020-11-10 10:13:37.873994,2020-11-10 10:13:38.643705,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,NaN,0.0
5,5,finished,Al,strain_0_98,/strain_0_98,archive/murn/murn_hdf5,2020-11-10 10:13:40.316294,2020-11-10 10:13:41.148311,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,NaN,0.0
6,6,finished,Al,strain_1_0,/strain_1_0,archive/murn/murn_hdf5,2020-11-10 10:13:42.488052,2020-11-10 10:13:43.235179,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,NaN,0.0
7,7,finished,Al,strain_1_02,/strain_1_02,archive/murn/murn_hdf5,2020-11-10 10:13:44.863735,2020-11-10 10:13:45.507020,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,NaN,0.0
8,8,finished,Al,strain_1_04,/strain_1_04,archive/murn/murn_hdf5,2020-11-10 10:13:46.731088,2020-11-10 10:13:47.445857,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,NaN,0.0
9,9,finished,Al,strain_1_06,/strain_1_06,archive/murn/murn_hdf5,2020-11-10 10:13:48.755317,2020-11-10 10:13:49.733949,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,NaN,0.0


In [10]:
directory_to_import_to = "/Users/jan/pyiron/projects/2020/2020-11-10-database-transfer/new/"

In [11]:
archive_directory = "/Users/jan/pyiron/projects/2020/2020-11-10-database-transfer/archive/"

In [12]:
import_jobs(directory_to_import_to=directory_to_import_to, archive_directory=archive_directory, df=df)

In [13]:
pr = Project(directory_to_import_to)

In [14]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,230,finished,Al,murn,/murn,/Users/jan/pyiron/projects/,2020/2020-11-10-database-transfer/new/murn,2020-11-10 10:13:29.972786,2020-11-10 10:14:00.727395,30.0,pyiron@MacBook-Pro.local#1#11/11,Murnaghan,0.3.0,None,NaN
1,231,finished,Al,strain_0_9,/strain_0_9,/Users/jan/pyiron/projects/,2020/2020-11-10-database-transfer/new/murn/murn_hdf5,2020-11-10 10:13:30.989205,2020-11-10 10:13:31.635240,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,None,230.0
2,232,finished,Al,strain_0_92,/strain_0_92,/Users/jan/pyiron/projects/,2020/2020-11-10-database-transfer/new/murn/murn_hdf5,2020-11-10 10:13:32.900224,2020-11-10 10:13:33.879257,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,None,230.0
3,233,finished,Al,strain_0_94,/strain_0_94,/Users/jan/pyiron/projects/,2020/2020-11-10-database-transfer/new/murn/murn_hdf5,2020-11-10 10:13:35.903589,2020-11-10 10:13:36.545234,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,None,230.0
4,234,finished,Al,strain_0_96,/strain_0_96,/Users/jan/pyiron/projects/,2020/2020-11-10-database-transfer/new/murn/murn_hdf5,2020-11-10 10:13:37.873994,2020-11-10 10:13:38.643705,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,None,230.0
5,235,finished,Al,strain_0_98,/strain_0_98,/Users/jan/pyiron/projects/,2020/2020-11-10-database-transfer/new/murn/murn_hdf5,2020-11-10 10:13:40.316294,2020-11-10 10:13:41.148311,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,None,230.0
6,236,finished,Al,strain_1_0,/strain_1_0,/Users/jan/pyiron/projects/,2020/2020-11-10-database-transfer/new/murn/murn_hdf5,2020-11-10 10:13:42.488052,2020-11-10 10:13:43.235179,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,None,230.0
7,237,finished,Al,strain_1_02,/strain_1_02,/Users/jan/pyiron/projects/,2020/2020-11-10-database-transfer/new/murn/murn_hdf5,2020-11-10 10:13:44.863735,2020-11-10 10:13:45.507020,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,None,230.0
8,238,finished,Al,strain_1_04,/strain_1_04,/Users/jan/pyiron/projects/,2020/2020-11-10-database-transfer/new/murn/murn_hdf5,2020-11-10 10:13:46.731088,2020-11-10 10:13:47.445857,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,None,230.0
9,239,finished,Al,strain_1_06,/strain_1_06,/Users/jan/pyiron/projects/,2020/2020-11-10-database-transfer/new/murn/murn_hdf5,2020-11-10 10:13:48.755317,2020-11-10 10:13:49.733949,0.0,pyiron@MacBook-Pro.local#1,Lammps,0.1,None,230.0
